In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
import numpy as np
from keras import backend as K
import tensorflow as tf
from tqdm import tqdm
from torchsummary import summary

Using TensorFlow backend.
/home/mobility/anaconda/envs/michael_env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [37]:
sequence_just_stops = np.load('../data/sentence_data/sequence_just_stops_window_16_with_categoricals.npy')

In [38]:
X = sequence_just_stops[0]
y = sequence_just_stops[1]

In [39]:
y = y.astype(int)

In [40]:
query = y==2
y[query]=1
query = y==3
y[query]=2
np.unique(y)

array([0, 1, 2])

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=1023)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train, test_size=0.2, random_state=1023)

In [42]:
X_train[0]

[array([[-8.05779846e+00, -3.48836469e+01,  1.30000000e+01,
          1.38888889e-01,  1.05430000e+04,  2.35243633e+02,
          5.00000000e+00,  1.00000000e+01],
        [-8.05848579e+00, -3.48816272e+01,  2.80000000e+01,
          5.55555556e-02,  1.07800000e+04,  1.83674266e+02,
          5.00000000e+00,  1.00000000e+01],
        [-8.05936922e+00, -3.48802162e+01,  2.20000000e+01,
          9.25925926e-03,  1.09700000e+04,  1.94964560e+02,
          5.00000000e+00,  1.00000000e+01],
        [-8.06037039e+00, -3.48787604e+01,  2.30000000e+01,
          3.70370370e-02,  1.11670000e+04,  1.48375492e+02,
          5.00000000e+00,  1.00000000e+01],
        [-8.06139707e+00, -3.48796269e+01,  1.90000000e+01,
          1.01851852e-01,  1.13270000e+04,  2.54136015e+02,
          5.00000000e+00,  1.00000000e+01],
        [-8.06337160e+00, -3.48808062e+01,  3.00000000e+01,
          0.00000000e+00,  1.15810000e+04,  0.00000000e+00,
          5.00000000e+00,  1.00000000e+01],
        [-8.0633

In [43]:
def get_batches(x, y,batch_size=32):
    list_x_b, list_x_a = list(), list()
    list_x_target, list_stat_b, list_stat_a = list(), list(), list()
    list_y = list()

    x1 = x
    num_classes = 3
    y = (torch.from_numpy(y_train.reshape((-1,1))) == torch.arange(num_classes).reshape(1, num_classes)).float()

    for i in range(0,len(x1),batch_size):
        if (i+batch_size)<len(x1):
            for j in range(i,i+batch_size):
                list_x_b.append(x1[j][0])
                list_x_a.append(x1[j][1])
                list_x_target.append(x1[j][2])
                list_stat_b.append(x1[j][3])
                list_stat_a.append(x1[j][4])
                list_y.append(y[j].data.numpy())
            yield [np.array(list_x_b)[:,:,:6],np.array(list_x_b)[:,:,6],np.array(list_x_b)[:,:,7],
                    np.array(list_x_a)[:,:,:6],np.array(list_x_a)[:,:,6],np.array(list_x_a)[:,:,7],
                    np.array(list_x_target)[:,:6],np.array(list_x_target)[:,6],np.array(list_x_target)[:,7],
                    np.array(list_stat_b),
                    np.array(list_stat_a)],np.array(list_y)
        list_x_b,list_x_a,list_x_target,list_stat_b,list_stat_a,list_y=[],[],[],[],[],[]


In [54]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Conv1d(6,32,16)
        self.conv2 = nn.Conv1d(6,32,16)
        self.fc1 = nn.Linear(110,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32, 3)
    
    def forward(self,x_before,x_after,x,x_before_stat,x_after_stat):
        x_before = F.relu(self.conv1(x_before))
        x_after = F.relu(self.conv1(x_after))
        
        x_before = x_before.view(-1,self.num_flat_features(x_before))
        x_after = x_after.view(-1,self.num_flat_features(x_after)) 
        
        concat = torch.cat([x_before_stat,x_before,x,x_after,x_after_stat],dim=1)
        
        result = F.relu(self.fc1(concat))
        result = F.relu(self.fc2(result))
        result = self.fc3(result)
        return result
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [55]:
net = ConvNet()
net = net.double()

In [56]:
print(net)

ConvNet(
  (conv1): Conv1d(6, 32, kernel_size=(16,), stride=(1,))
  (conv2): Conv1d(6, 32, kernel_size=(16,), stride=(1,))
  (fc1): Linear(in_features=110, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=3, bias=True)
)


In [57]:
model_parameters = filter(lambda p: p.requires_grad, net.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

15491

In [58]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [59]:
net = net.cuda()

In [60]:
a = get_batches(X_train, y_train)

In [61]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [66]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        #Get training data
        train_loader = get_batches(X_train,y_train,batch_size)
        n_batches = len(X_train)//batch_size
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        total_val_loss = 0
        total_acc = 0
        partial_acc = 0
        
        number_of_subepoch = 0
        
        for i, data in enumerate(train_loader, 0):
            
            inputs, labels = data
            
            input_x_before = Variable(torch.from_numpy(data[0][0].reshape(-1,6,16)).clone())
            input_x_after = Variable(torch.from_numpy(data[0][3].reshape(-1,6,16)).clone())
            input_x = Variable(torch.from_numpy(data[0][6]).clone())
            input_x_before_stat = Variable(torch.from_numpy(data[0][9]).clone())
            input_x_after_stat = Variable(torch.from_numpy(data[0][10]).clone())
            
            
            #Forward pass, backward pass, optimize
            outputs = net(input_x_before.cuda(),
                          input_x_after.cuda(),
                          input_x.cuda(),
                          input_x_before_stat.cuda(),
                          input_x_after_stat.cuda())
            
            labels = np.argmax(labels, axis=1)
            loss_size = loss(outputs, Variable(torch.from_numpy(labels).clone().long(),requires_grad=False).cuda())
            loss_size.backward()
            optimizer.step()
            
            total_val_loss+= loss_size.item()
            partial_acc += torch.sum(torch.argmax(outputs, dim=1) == torch.from_numpy(labels).cuda()).item()/batch_size
            number_of_subepoch+=1

        print(f'loss:{total_val_loss/number_of_subepoch}')
        #At the end of the epoch, do a pass on the validation set
        val_batch = get_batches(X_val,y_val,32)
        with torch.no_grad():
            for i, data in enumerate(val_batch,0):
                
                inputs1,labels1 = data
                input_x_before = Variable(torch.from_numpy(data[0][0].reshape(-1,6,16)).clone())
                input_x_after = Variable(torch.from_numpy(data[0][3].reshape(-1,6,16)).clone())
                input_x = Variable(torch.from_numpy(data[0][6]).clone())
                input_x_before_stat = Variable(torch.from_numpy(data[0][9]).clone())
                input_x_after_stat = Variable(torch.from_numpy(data[0][10]).clone())
                
                #Forward pass
                val_outputs = net(input_x_before.cuda(),
                                  input_x_after.cuda(),
                                  input_x.cuda(),
                                  input_x_before_stat.cuda(),
                                  input_x_after_stat.cuda())
                val_loss_size = loss(val_outputs, Variable(torch.from_numpy(labels).clone()).cuda())
                total_val_loss += val_loss_size.item()
            
        print("Validation loss = {:.2f}".format(total_val_loss / i))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))


In [ ]:
trainNet(net,32,20,0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 20
learning_rate= 0.001
loss:0.8751320908853254
Validation loss = 4.42
loss:0.8688843015198097
Validation loss = 4.35
loss:0.8643170429803723
Validation loss = 4.33
loss:0.8677671540239374
Validation loss = 4.38
loss:0.8626736543187787
Validation loss = 4.33
loss:0.8618058074090098
Validation loss = 4.33
loss:0.8587921571848647
Validation loss = 4.31
loss:0.8653078312403032
Validation loss = 4.34
loss:0.8618708369961452
Validation loss = 4.35
loss:0.860145820843664
Validation loss = 4.32


In [ ]:
a = torch.rand(5,5)

In [ ]:
a.size()[0]

In [ ]:
a = 5

In [ ]:
a/= 2

In [ ]:
a